# Stock Analysis with Vertex AI Integration

This notebook demonstrates how to combine stock market analysis with Vertex AI's powerful language models. We'll create a smart stock analysis system that can:

1. Fetch and analyze stock data
2. Generate visualizations
3. Use Vertex AI to provide natural language insights
4. Create automated reports

## Setup
First, let's install and import the required packages.

In [ ]:
# Dependencies are managed through requirements.txt
# No need for pip install commands here

In [ ]:
# Import required libraries
import os
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from google.cloud import aiplatform
from vertexai.generative_models import GenerativeModel

# Set up plotting style
plt.style.use('seaborn')
sns.set_palette("husl")

## Initialize Vertex AI
Now let's set up our connection to Vertex AI using the credentials we configured earlier.

In [ ]:
# Initialize Vertex AI with our project settings
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    'key.json',
    scopes=['https://www.googleapis.com/auth/cloud-platform']
)

# Initialize Vertex AI
project_id = 'wmt-us-gg-shrnk-prod'
location = 'us-central1'
aiplatform.init(
    project=project_id,
    location=location,
    credentials=credentials
)

# Initialize the Gemini model
model = GenerativeModel("gemini-2.0-flash-001")

## Stock Data Functions
Let's create functions to fetch and analyze stock data.

In [ ]:
def fetch_stock_data(symbol, period='1y'):
    """Fetch stock data using yfinance."""
    stock = yf.Ticker(symbol)
    df = stock.history(period=period)
    return df

def calculate_metrics(df):
    """Calculate key stock metrics."""
    metrics = {
        'current_price': df['Close'][-1],
        'daily_return': df['Close'].pct_change().mean(),
        'volatility': df['Close'].pct_change().std(),
        'volume': df['Volume'].mean(),
        'high_52w': df['High'].max(),
        'low_52w': df['Low'].min()
    }
    return metrics

def analyze_with_ai(metrics, symbol):
    """Use Vertex AI to analyze stock metrics."""
    prompt = f"""
    Analyze these stock metrics for {symbol}:
    Current Price: ${metrics['current_price']:.2f}
    Daily Return: {metrics['daily_return']*100:.2f}%
    Volatility: {metrics['volatility']*100:.2f}%
    Average Volume: {metrics['volume']:,.0f}
    52-Week High: ${metrics['high_52w']:.2f}
    52-Week Low: ${metrics['low_52w']:.2f}
    
    Provide a brief analysis including:
    1. Key observations
    2. Notable trends
    3. Potential risks or opportunities
    """
    
    chat = model.start_chat()
    response = chat.send_message(prompt)
    return response.text

## Test the Integration
Let's test our functions with a sample stock (Walmart).

In [ ]:
# Fetch Walmart stock data
symbol = 'WMT'
df = fetch_stock_data(symbol)

# Calculate metrics
metrics = calculate_metrics(df)

# Display basic metrics
print(f"Stock Analysis for {symbol}:")
for key, value in metrics.items():
    print(f"{key}: {value:,.2f}")

In [ ]:
# Create a price chart
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['Close'])
plt.title(f'{symbol} Stock Price - Past Year')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.grid(True)
plt.show()

# Get AI analysis
print("\nAI Analysis:")
analysis = analyze_with_ai(metrics, symbol)
print(analysis)

## Export to Quarto
Now let's create a function to export our analysis to a Quarto document.

In [ ]:
def create_quarto_report(symbol, metrics, analysis, df):
    """Create a Quarto report with stock analysis."""
    report_content = f"""---
title: "Stock Analysis Report: {symbol}"
format:
  html:
    theme: cosmo
    toc: true
date: "{datetime.now().strftime('%Y-%m-%d')}"
---

# Stock Analysis Report for {symbol}

## Key Metrics

- Current Price: ${metrics['current_price']:.2f}
- Daily Return: {metrics['daily_return']*100:.2f}%
- Volatility: {metrics['volatility']*100:.2f}%
- Average Volume: {metrics['volume']:,.0f}
- 52-Week High: ${metrics['high_52w']:.2f}
- 52-Week Low: ${metrics['low_52w']:.2f}

## AI Analysis

{analysis}

## Price Chart

```{{python}}
#| label: price-chart
#| fig-cap: "Stock Price Over Time"
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(df.index, df['Close'])
plt.title('{symbol} Stock Price - Past Year')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.grid(True)
plt.show()
```
"""
    
    # Save the report
    with open(f'stock_report_{symbol}.qmd', 'w') as f:
        f.write(report_content)
    
    return f'stock_report_{symbol}.qmd'

# Create the report
report_file = create_quarto_report(symbol, metrics, analysis, df)
print(f"Report created: {report_file}")
print("You can now render this report in Posit using the Quarto tools.")